# SiRF Scanner

## Written by Michael George

Iterate through folders, looking for file reader issues caused by wrong extension, missing headers, bad checksums, etc.

In [1]:
import os
import sys
import traceback

readersPath = os.path.join('.', 'readers')
if readersPath not in sys.path:
    sys.path.extend([readersPath])

from generic_reader import getReader

## Main Function

In [2]:
def scanFiles():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions', 'broken')

    exceptions = {}
    passes = []

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ('.sbn', '.sbp'):
                filePath = os.path.join(root, file)
                reader = getReader(filePath)
                try:
                    reader.load()
                    passes.append(file)
                    print('.', end='')

                except Exception:
                    exceptions[file] = traceback.format_exc()
                    print('E', end='')

    print(os.linesep * 2 + 'Errors:')
    if len(exceptions) > 0:
        for file in exceptions:
            print(os.linesep + file)
            print(exceptions[file].split(os.linesep)[-2:-1])
    else:
        print(os.linesep + 'None!')

    print(os.linesep * 2 + 'Passes:' + os.linesep)
    if len(passes) > 0:
        for file in passes:
            print(file)
    else:
        print('None!')

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], ".."))

    scanFiles()

EEEEEEEEE

Errors:

TONY_1002841_20090718_120329_DLG.SBP
['ValueError: hour must be in 0..23']

SIMON R_932000407_20120517_194953_DLG.SBP
['ValueError: Mismatch of seconds and timestamps']

ANNETTE_1001984_20100513_192054_DLG.SBP
['ValueError: hour must be in 0..23']

PHIL_1001781_20100603_203612_DLG.SBP
['ValueError: hour must be in 0..23']

ANDRE_1003053_20100502_221354_DLG.SBP
['ValueError: hour must be in 0..23']

PHIL_1001781_20100610_202530_DLG.SBP
['ValueError: hour must be in 0..23']

TONY_1002841_20100610_201619_DLG.SBP
['ValueError: hour must be in 0..23']

CHRIS_BATES.sbp
["ValueError: invalid literal for int() with base 10: 'unknown'"]

ANDRE_1003053_20100506_185459_DLG.SBP
['ValueError: hour must be in 0..23']


Passes:

None!
